Carregando o df. O df escolhido foi "df_corr_ten" , pois testamos o modelo com  todos os df's criados no colab anterior e o que obteve as melhores métricas foi o "df_corr_ten"

In [1]:
!gdown 1Kv699tEjtwTqqk0m12R-dGyrJ6SqJi3Q


Downloading...
From: https://drive.google.com/uc?id=1Kv699tEjtwTqqk0m12R-dGyrJ6SqJi3Q
To: /content/df_corr_ten.csv
100% 138k/138k [00:00<00:00, 83.2MB/s]


In [2]:
!pip install shap
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 KB 21.5 MB/s eta 0:00:00


In [3]:
df = pd.read_csv('/content/df_corr_ten.csv')

In [4]:
#removendo colunas que não entrarão no modelo
#as colunas derivadas de hormonioterapia não podem entrar pois elas antecipam qual será o tratamento
df = df.drop(columns = ['Unnamed: 0', 'hormonioterapia_hormonioterapia_Adjuvante',
       'hormonioterapia_hormonioterapia_Neoadjuvante',
       'hormonioterapia_hormonioterapia_Não realizou',
       'hormonioterapia_hormonioterapia_Outras combinacoes', 'record_id','radioterapia'])

In [5]:
df.shape

(2048, 13)

# 6 Criação de modelo e definição de métricas para testes 

## 6.1 Data Split

Dividiremos o Dataframe em conjunto de treino e teste. Note que no modelo final não há necessidade , tendo em vista que usaremos todos os dados serão utilizados para treino.

In [6]:
#Separando o conjunto de treino e teste
from sklearn.model_selection import train_test_split

X = df.drop(columns = 'tratamento')
y = df['tratamento']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 51)

In [7]:
X.isnull().mean()

idade_primeiro_diagnostico                 0.0
grupo_estadio_clinico                      0.0
classificacao_tnm_clinico_t                0.0
classificacao_tnm_clinico_n                0.0
subtipo_tumoral                            0.0
indice_h_receptorde_progesterona           0.0
ki67_pct                                   0.0
receptor_estrogenio                        0.0
receptor_progesterona                      0.0
receptor_progesterona_quantificacao_pct    0.0
receptorde_estrogenio_quantificacao_pct    0.0
ki67_maior14pct                            0.0
dtype: float64

## 6.2 Normalização 

A normalização consiste em deixar todas as colunas em uma escala comum de valores. É importante faze-la pois: reduz a redundância dos dados, reduz as inconsistências e erros nos dados, melhora o desempenho do modelo.

In [8]:
#Normalizando os conjutnos
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)


## 6.2.1 Salvando normalização

In [9]:
import joblib
filename = 'normalizacao.sav'
joblib.dump(scaler, filename)
  
normalizado = joblib.load(filename)

## 6.3 CRIAÇÃO DOS MODELOS E COLOCAÇÃO DAS MÉTRICAS

####6.3.1 Stacking Classifier

O modelo que obteve o melhor desempenho foi o Stacking Classifier, que basicamente combina alguns modelos de classificação para criar uma nova predição melhor e mais apurada. Logo ele foi selecionado.

In [10]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

clf1 = KNeighborsClassifier()
clf2 = GaussianNB()
clf3 = LogisticRegression()
clf4 = RandomForestClassifier()
clf5 = svm.SVC()

# cria o stacking classifier
estimators = [('knn', clf1), ('nb', clf2), ('rf', clf4),('Lr',clf3), ('svm', clf5)]
stacking_clf = StackingClassifier(estimators=estimators, final_estimator=clf5)

# treina o modelo
stacking_clf.fit(X, y)




#abaixo teremos as métricas que utilizamos, comentaremos isso pois agora não estamos mais utilizando o X_train X_test Y_train Y_test. Utilizaremos apenas X e Y
# avalia o modelo
"""print(classification_report(y_test, y_pred))


# AUC - ROC
auc_roc = roc_auc_score(y_test, y_pred)
print("AUC ROC:", auc_roc)

cm = confusion_matrix(y_test, y_pred)

sns.heatmap(cm, annot=True, fmt='g', cmap='Blues')
plt.xlabel('Previsão')
plt.ylabel('Realidade')
plt.show()"""

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

'print(classification_report(y_test, y_pred))\n\n\n# AUC - ROC\nauc_roc = roc_auc_score(y_test, y_pred)\nprint("AUC ROC:", auc_roc)\n\ncm = confusion_matrix(y_test, y_pred)\n\nsns.heatmap(cm, annot=True, fmt=\'g\', cmap=\'Blues\')\nplt.xlabel(\'Previsão\')\nplt.ylabel(\'Realidade\')\nplt.show()'

####6.3.5 Logistic Regression

## 7 Salvando o modelo:

O código abaixo usa joblib para para salvar o modelo 

In [11]:
import joblib
filename = 'modelo_stacking_clf.sav'
joblib.dump(stacking_clf, filename)
  
loaded_model = joblib.load(filename)
print(df.columns)

Index(['idade_primeiro_diagnostico', 'tratamento', 'grupo_estadio_clinico',
       'classificacao_tnm_clinico_t', 'classificacao_tnm_clinico_n',
       'subtipo_tumoral', 'indice_h_receptorde_progesterona', 'ki67_pct',
       'receptor_estrogenio', 'receptor_progesterona',
       'receptor_progesterona_quantificacao_pct',
       'receptorde_estrogenio_quantificacao_pct', 'ki67_maior14pct'],
      dtype='object')


In [12]:
!pip install skl2onnx


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.0/294.0 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 99.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 KB 12.5 MB/s eta 0:00:00


In [13]:
import pickle
from sklearn.pipeline import Pipeline
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# Salve o modelo em um arquivo .pkl
with open('modelo.pkl', 'wb') as f:
    pickle.dump(stacking_clf, f)



In [14]:
import pickle
from sklearn.pipeline import Pipeline
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

In [15]:
# Carrega o modelo pré-treinado com Scikit-Learn
with open('modelo.pkl', 'rb') as f:
    model = pickle.load(f)

# Converte o modelo para um objeto compatível com o ML.NET
input_type = [('input', FloatTensorType([None, X_train.shape[1]]))]
onnx_model = convert_sklearn(model, 'modeloFINAL', initial_types=input_type)

# Salva o modelo em formato ONNX
with open('modeloFINAL.onnx', 'wb') as f:
    f.write(onnx_model.SerializeToString())